In [6]:
# https://platform.olimpiada-ai.ro/problems/29

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [7]:
train = pd.read_csv("/kaggle/input/classify-text-into-categories/train.csv")
test = pd.read_csv("/kaggle/input/classify-text-into-categories/test.csv")

In [8]:
class2idx = {
    'WELLNESS': 0,
    'ENTERTAINMENT': 1,
    'POLITICS': 2
}

idx2class = {v: k for k, v in class2idx.items()}

train['label'] = train['label'].map(class2idx.get)

In [10]:
from sklearn.model_selection import train_test_split
from catboost import Pool

features = ['text']
text_features = ['text']

X, y = train[features], train['label']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=42, test_size=0.2)

train_pool = Pool(X_train, y_train, text_features=text_features)
valid_pool = Pool(X_valid, y_valid, text_features=text_features)

In [16]:
from catboost import CatBoostClassifier

params = {
    'iterations': 10,
    'loss_function': 'MultiClass',
    'max_depth': 6,
    'eval_metric': 'TotalF1:average=Macro',
    'metric_period': 1
}

model = CatBoostClassifier(**params)

model.fit(train_pool, eval_set=valid_pool)

Learning rate set to 0.5
0:	learn: 0.9129296	test: 0.9236440	best: 0.9236440 (0)	total: 419ms	remaining: 3.77s
1:	learn: 0.9108235	test: 0.9243751	best: 0.9243751 (1)	total: 809ms	remaining: 3.23s
2:	learn: 0.9108425	test: 0.9244591	best: 0.9244591 (2)	total: 1.19s	remaining: 2.77s
3:	learn: 0.9108378	test: 0.9249165	best: 0.9249165 (3)	total: 1.56s	remaining: 2.34s
4:	learn: 0.9118817	test: 0.9250174	best: 0.9250174 (4)	total: 1.94s	remaining: 1.94s
5:	learn: 0.9128809	test: 0.9251194	best: 0.9251194 (5)	total: 2.38s	remaining: 1.59s
6:	learn: 0.9142344	test: 0.9251877	best: 0.9251877 (6)	total: 2.75s	remaining: 1.18s
7:	learn: 0.9147179	test: 0.9250290	best: 0.9251877 (6)	total: 3.12s	remaining: 780ms
8:	learn: 0.9149332	test: 0.9253618	best: 0.9253618 (8)	total: 3.45s	remaining: 383ms
9:	learn: 0.9155689	test: 0.9260461	best: 0.9260461 (9)	total: 3.81s	remaining: 0us

bestTest = 0.9260461031
bestIteration = 9



In [18]:
from sklearn.metrics import f1_score

y_pred = model.predict(X_valid).flatten()
score = f1_score(y_valid, y_pred, average='macro')
print(f"F1 Score: {score:.5f}")

F1 Score: 0.92605


In [21]:
X_test = test[features]

y_pred = model.predict(X_test).flatten()

subm = pd.DataFrame({
    'SampleID': test['SampleID'],
    'label': list(map(idx2class.get, y_pred))
})

subm.head()

,SampleID,label
0,103811,ENTERTAINMENT
1,207637,WELLNESS
2,151717,WELLNESS
3,27926,POLITICS
4,115640,POLITICS


In [24]:
subm.to_csv("submission.csv", index=False)